In [ ]:
import zipfile
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# Step 1: Extract ZIP File
zip_file_path = '/content/Boerhavia.zip'
extraction_dir = 'dataset'

os.makedirs(extraction_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

print("Extraction completed!")

Extraction completed!


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Paths to the dataset
train_dir = "/content/dataset/Boerhavia/train"
valid_dir = "/content/dataset/Boerhavia/valid"
test_dir = "/content/dataset/Boerhavia/test"

# Parameters
img_height, img_width = 180, 180  # Input image size
batch_size = 32
num_classes = 2  # B.diffusa and B.ecreta

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,  # Increase rotation
    width_shift_range=0.3,  # Increase width shift
    height_shift_range=0.3,  # Increase height shift
    shear_range=0.3,  # Increase shear
    zoom_range=0.3,  # Increase zoom
    horizontal_flip=True,
    vertical_flip=True,  # Include vertical flip
    fill_mode="nearest",
)


valid_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
)

valid_generator = valid_test_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
)

test_generator = valid_test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
)

# Model Definition
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(num_classes, activation="softmax"),
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Callbacks
checkpoint = ModelCheckpoint(
    "Boerhavia_Model.keras",  # Change the file extension to .keras
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1,
)

early_stopping = EarlyStopping(monitor="val_loss", patience=5, verbose=1)

# Training
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=valid_generator,
    callbacks=[checkpoint, early_stopping],
)

# Evaluation
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save the model
model.save("Boerhavia_Model_Final.keras")  # Change the file extension to .keras


Found 759 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Found 98 images belonging to 2 classes.
Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5549 - loss: 1.0180
Epoch 1: val_accuracy improved from -inf to 0.84167, saving model to Boerhavia_Model.keras
24/24 ━━━━━━━━━━━━━━━━━━━━ 68s 3s/step - accuracy: 0.5582 - loss: 1.0076 - val_accuracy: 0.8417 - val_loss: 0.3642
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8273 - loss: 0.4293
Epoch 2: val_accuracy did not improve from 0.84167
24/24 ━━━━━━━━━━━━━━━━━━━━ 63s 3s/step - accuracy: 0.8271 - loss: 0.4295 - val_accuracy: 0.7667 - val_loss: 0.4731
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8341 - loss: 0.3865
Epoch 3: val_accuracy did not improve from 0.84167
24/24 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.8338 - loss: 0.3874 - val_accuracy: 0.7917 - val_loss: 0.4020
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8494 - loss: 0.3569
Epoch 4: